In [140]:
import fastText #
import pandas as pd
import os
from fastText import train_supervised
import numpy as np

In [7]:
# To show the output of all lines in a cell rather that just the last line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

`The architecture of fasttext supervised learning is like word2vec CBOW where the target word is replaced with the label.`

In [8]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [9]:
DATA_DIR = "~/Downloads/sentiment_labelled_sentences/"

In [380]:
data_amazon = pd.read_table(DATA_DIR+"amazon_cells_labelled.txt"
                          , sep='\t', header=None, names=['sentence', 'sentiment'])
data_imdb = pd.read_table(DATA_DIR+"imdb_labelled.txt"
                          , sep='\t', header=None, names=['sentence', 'sentiment'])
data_yelp = pd.read_table(DATA_DIR+"yelp_labelled.txt"
                          , sep='\t', header=None, names=['sentence', 'sentiment'])

In [381]:
all_data = pd.concat([data_amazon, data_imdb, data_yelp])
all_data.head(2)

,sentence,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1


### Formatting the label as the default for fasttext

In [382]:
all_data['label'] = all_data.apply(lambda row: '__label__' + str(row.sentiment), axis=1)

In [383]:
all_data.head(2)

,sentence,sentiment,label
0,So there is no way for me to plug it in here i...,0,__label__0
1,"Good case, Excellent value.",1,__label__1


In [384]:
import re

def normalize(row):
    lower = row['sentence'].lower()
    #correct all multiple white spaces to a single white space
    no_mult_ws = re.sub('[\s]+', ' ', lower)
    text = no_mult_ws.strip()
    return text

all_data['normalized_sentence'] = all_data.apply(normalize, axis=1)

In [385]:
all_data.head(2)

,sentence,sentiment,label,normalized_sentence
0,So there is no way for me to plug it in here i...,0,__label__0,so there is no way for me to plug it in here i...
1,"Good case, Excellent value.",1,__label__1,"good case, excellent value."


In [386]:
labeled_data = all_data.drop(['sentence', 'sentiment'], axis=1)
labeled_data.head(2)

,label,normalized_sentence
0,__label__0,so there is no way for me to plug it in here i...
1,__label__1,"good case, excellent value."


In [387]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(labeled_data, test_size=0.35, random_state=4)

In [388]:
test_data.head(2)

,label,normalized_sentence
177,__label__1,"the atmosphere is modern and hip, while mainta..."
720,__label__1,"cute, quaint, simple, honest."


#### Save model to disk to be read by fasttext

In [389]:
labeled_data.to_csv(path_or_buf='./sentiment.all', header=False, index=False, sep='\t')
train_data.to_csv(path_or_buf='./sentiment.train', header=False, index=False, sep='\t')
test_data.to_csv(path_or_buf='./sentiment.test', header=False, index=False, sep='\t')

### Fasttext model training/eval/etc.
##### This model is a binary classifier.

In [220]:
train_data_path = os.path.join("./", 'sentiment.train')
test_data_path = os.path.join("./", 'sentiment.test')

sentiment_model = train_supervised(
    input=train_data_path,
    lr=1,
    dim=100,
    ws=5,
    epoch=5,
    minCount=1,
    minCountLabel=0,
    minn=2,
    maxn=3,
    neg=5,
    wordNgrams=2,
    loss="softmax",
    bucket=200000,
    lrUpdateRate=100,
    t=1e-4,
    label="__label__",
    verbose=2,
    pretrainedVectors="",
)
print("On train")
print_results(*sentiment_model.test(train_data_path))
print("On test")
print_results(*sentiment_model.test(test_data_path))
sentiment_model.save_model("sentiment_model.bin")

On train
N	1786
P@1	0.933
R@1	0.933
On test
N	962
P@1	0.790
R@1	0.790


Check here for the input arguments explanations: 'https://fasttext.cc/docs/en/options.html'

In [316]:
# Testing trained model on some random document
sentiment_model.predict("you are not a cool guy but i really like you", k=2)
sentiment_model.predict("yeah..", k=2)

(('__label__0', '__label__1'), array([ 0.54495442,  0.45506558]))

(('__label__1', '__label__0'), array([ 0.75722229,  0.24279772]))

In [ ]:
sentiment_model.predict("you are not a cool guy but i really like you", k=2)
sentiment_model.predict("yeah..", k=2)

Quantizing sacrifices a bit of performance to reduce the size of model. Good when size matters, like when deploying to
edge device, e.g. mobile.

In [74]:
sentiment_model.quantize(input=train_data_path, qnorm=True, retrain=True, cutoff=200000)
sentiment_model.save_model("sent_model.ftz")

In [75]:
print_results(*sentiment_model.test(test_data_path))

N	962
P@1	0.802
R@1	0.802


In [76]:
sentiment_model.predict("you are not a cool guy but i really like you", k=2)
sentiment_model.predict("yeah..", k=2)

(('__label__1', '__label__0'), array([ 0.65207916,  0.34794083]))

(('__label__1', '__label__0'), array([ 0.84964049,  0.15037954]))

In [256]:
# pr = sentiment_model.predict("you are not a cool guy but i really like you", k=1)
pr = sentiment_model.predict("nice", k=1)
pr

(('__label__1',), array([ 1.00000989]))

In [258]:
if "0" in pr[0][0]:
    print(1 - pr[1][0])
else:
    pr[1][0]

1.0000098943710327

In [262]:
def get_pos_probab(doc):
    pr = sentiment_model.predict(doc, k=1)
    if "0" in pr[0][0]:
        return round(1 - pr[1][0], 2)
    else:
        return round(pr[1][0], 2)

In [263]:
test_data['probability'] = test_data['normalized_sentence'].apply(get_pos_probab)
test_data['prediction'] = test_data['probability'].apply(lambda row: int(round(row, 0)))
test_data['orig_label'] = test_data['label'].apply(lambda row: row[-1:])

In [265]:
test_data.head(5)

,label,normalized_sentence,probability,prediction,orig_label
177,__label__1,"the atmosphere is modern and hip, while mainta...",0.79,1,1
720,__label__1,"cute, quaint, simple, honest.",0.48,0,1
525,__label__0,"it's an empty, hollow shell of a movie.",0.81,1,0
630,__label__0,don't bother - go to the store.,0.06,0,0
137,__label__0,to those who find this movie intelligent or ev...,0.79,1,0


In [283]:
pd.options.mode.chained_assignment = None

In [284]:
train_data['probability'] = train_data['normalized_sentence'].apply(get_pos_probab)
train_data['prediction'] = train_data['probability'].apply(lambda row: int(round(row, 0)))
train_data['orig_label'] = train_data['label'].apply(lambda row: row[-1:])

In [267]:
train_data.head(3)

,label,normalized_sentence,probability,prediction,orig_label
15,__label__1,highly recommend for any one who has a blue to...,0.79,1,1
356,__label__0,"sadly, gordon ramsey's steak is a place we sha...",0.42,0,0
496,__label__1,the pancake was also really good and pretty la...,0.79,1,1


In [268]:
train_data.dtypes
train_data['orig_label'] = train_data['orig_label'].astype('int')
train_data['prediction'] = train_data['prediction'].astype('int')
train_data.dtypes

label                   object
normalized_sentence     object
probability            float64
prediction               int64
orig_label              object
dtype: object

label                   object
normalized_sentence     object
probability            float64
prediction               int64
orig_label               int64
dtype: object

In [277]:
test_data['orig_label'] = test_data['orig_label'].astype('category')
test_data['prediction'] = test_data['prediction'].astype('category')

In [278]:
test_data.head(6)
train_data.head(6)

,label,normalized_sentence,probability,prediction,orig_label
177,__label__1,"the atmosphere is modern and hip, while mainta...",0.79,1,1
720,__label__1,"cute, quaint, simple, honest.",0.48,0,1
525,__label__0,"it's an empty, hollow shell of a movie.",0.81,1,0
630,__label__0,don't bother - go to the store.,0.06,0,0
137,__label__0,to those who find this movie intelligent or ev...,0.79,1,0
555,__label__0,i know this is not like the other restaurants ...,0.29,0,0


,label,normalized_sentence,probability,prediction,orig_label
15,__label__1,highly recommend for any one who has a blue to...,0.79,1,1
356,__label__0,"sadly, gordon ramsey's steak is a place we sha...",0.42,0,0
496,__label__1,the pancake was also really good and pretty la...,0.79,1,1
753,__label__1,cheap but hey it works.. was pleasantly supris...,0.63,1,1
9,__label__1,a great touch.,1.00,1,1
975,__label__1,it is the best charger i have seen on the mark...,0.95,1,1


In [282]:
from sklearn import metrics

metrics.recall_score(train_data['orig_label'], train_data['prediction'], average='micro')
metrics.recall_score(test_data['orig_label'], test_data['prediction'], average='micro')
metrics.accuracy_score(test_data['orig_label'], test_data['prediction'])
metrics.recall_score(test_data['orig_label'], test_data['prediction'], average='macro')

0.93337066069428887

0.79106029106029108

0.79106029106029108

0.79107883817427394

However, in our example, we are going to use the precision@1 provided by fasttext itself, although the above cells show how to compute any other arbitrary metric.

Model parameter search/tuning..

In [394]:
train_data_path = os.path.join("./", 'sentiment.train')
test_data_path = os.path.join("./", 'sentiment.test')

def grid_search(lr, dim, ws, epoch, minn, maxn, wordNgrams):
    for l_rate in lr:
        for d in dim:
            for s in ws:
                for ep in epoch:
                    for mi in minn:
                        for ma in maxn:
                            if (ma >= mi):
                                for n in wordNgrams:
                                    sentiment_model_mc = train_supervised(
                                                        input=train_data_path,
                                                        lr=l_rate,
                                                        dim=d,
                                                        ws=s,
                                                        epoch=ep,
                                                        minCount=1,
                                                        minCountLabel=0,
                                                        minn=mi,
                                                        maxn=ma,
                                                        neg=5,
                                                        wordNgrams=n,
                                                        loss='softmax', # ns, hs, softmax, ova (for multilabel classification)
                                                        bucket=200000,
                                                        lrUpdateRate=100,
                                                        t=1e-4,
                                                        label="__label__",
                                                        verbose=2,
                                                        pretrainedVectors="",
                                                    )
                                    train_res = round(sentiment_model_mc.test(train_data_path)[1], 2) #precision
                                    test_res = round(sentiment_model_mc.test(test_data_path)[1], 2)
                                    ratio = train_res/test_res
                                    if (ratio > 0.95) & (ratio < 1.06) & (train_res > 0.8):
                                        print("{}, {}: *** lr = {}, dim = {}, ws = {}, epoch = {}, minn = {}, maxn = {}, wordNgrams = {}".format(train_res, test_res, l_rate, d, s, ep, mi, ma, n))
                                    else:
                                        print("{}, {}:     lr = {}, dim = {}, ws = {}, epoch = {}, minn = {}, maxn = {}, wordNgrams = {}".format(train_res, test_res, l_rate, d, s, ep, mi, ma, n))
                                

In [ ]:
# loss='softmax', minCount=1, neg=5

In [299]:
grid_search(lr=[0.9, 0.95], dim=[5, 10, 20, 40], ws=[4, 5, 6], 
            epoch=[1, 2, 3], minn=[2, 3, 4], maxn=[3, 4, 5, 6], wordNgrams=[1, 2, 3])

0.79, 0.75: *** lr = 0.9, dim = 5, ws = 4, epoch = 1, minn = 2, maxn = 3, wordNgrams = 1
0.81, 0.75:     lr = 0.9, dim = 5, ws = 4, epoch = 1, minn = 2, maxn = 3, wordNgrams = 2
0.82, 0.75:     lr = 0.9, dim = 5, ws = 4, epoch = 1, minn = 2, maxn = 3, wordNgrams = 3
0.81, 0.75:     lr = 0.9, dim = 5, ws = 4, epoch = 1, minn = 2, maxn = 4, wordNgrams = 1
0.81, 0.75:     lr = 0.9, dim = 5, ws = 4, epoch = 1, minn = 2, maxn = 4, wordNgrams = 2
0.82, 0.76:     lr = 0.9, dim = 5, ws = 4, epoch = 1, minn = 2, maxn = 4, wordNgrams = 3
0.82, 0.76:     lr = 0.9, dim = 5, ws = 4, epoch = 1, minn = 2, maxn = 5, wordNgrams = 1
0.81, 0.74:     lr = 0.9, dim = 5, ws = 4, epoch = 1, minn = 2, maxn = 5, wordNgrams = 2
0.81, 0.75:     lr = 0.9, dim = 5, ws = 4, epoch = 1, minn = 2, maxn = 5, wordNgrams = 3
0.83, 0.76:     lr = 0.9, dim = 5, ws = 4, epoch = 1, minn = 2, maxn = 6, wordNgrams = 1
0.8, 0.75:     lr = 0.9, dim = 5, ws = 4, epoch = 1, minn = 2, maxn = 6, wordNgrams = 2
0.81, 0.74:     lr = 0

In [300]:
grid_search(lr=[0.95, 1], dim=[5, 20], ws=[5, 6], 
            epoch=[1, 2], minn=[3,4], maxn=[3, 4, 5, 6], wordNgrams=[1, 2, 3])

0.85, 0.77:     lr = 0.95, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 3, wordNgrams = 1
0.85, 0.77:     lr = 0.95, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 3, wordNgrams = 2
0.87, 0.77:     lr = 0.95, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 3, wordNgrams = 3
0.84, 0.76:     lr = 0.95, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 4, wordNgrams = 1
0.84, 0.77:     lr = 0.95, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 4, wordNgrams = 2
0.85, 0.76:     lr = 0.95, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 4, wordNgrams = 3
0.85, 0.78:     lr = 0.95, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 5, wordNgrams = 1
0.85, 0.78:     lr = 0.95, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 5, wordNgrams = 2
0.85, 0.78:     lr = 0.95, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 5, wordNgrams = 3
0.85, 0.78:     lr = 0.95, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 6, wordNgrams = 1
0.85, 0.78:     lr = 0.95, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 6, wordNgrams = 2
0.84, 0.78

In [392]:
grid_search(lr=[0.05, 0.1], dim=[5, 20], ws=[5, 6], 
            epoch=[1, 2], minn=[3,4], maxn=[3, 4, 5, 6], wordNgrams=[1, 2, 3])

0.57, 0.58:     lr = 0.05, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 3, wordNgrams = 1
0.58, 0.56:     lr = 0.05, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 3, wordNgrams = 2
0.57, 0.56:     lr = 0.05, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 3, wordNgrams = 3
0.56, 0.57:     lr = 0.05, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 4, wordNgrams = 1
0.56, 0.56:     lr = 0.05, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 4, wordNgrams = 2
0.56, 0.56:     lr = 0.05, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 4, wordNgrams = 3
0.54, 0.56:     lr = 0.05, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 5, wordNgrams = 1
0.55, 0.56:     lr = 0.05, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 5, wordNgrams = 2
0.55, 0.56:     lr = 0.05, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 5, wordNgrams = 3
0.55, 0.54:     lr = 0.05, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 6, wordNgrams = 1
0.55, 0.55:     lr = 0.05, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 6, wordNgrams = 2
0.55, 0.55

In [395]:
grid_search(lr=[0.5, 2], dim=[5, 20], ws=[5, 6], 
            epoch=[1, 2], minn=[3,4], maxn=[3, 4, 5, 6], wordNgrams=[1, 2, 3])

0.74, 0.71:     lr = 0.5, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 3, wordNgrams = 1
0.71, 0.67:     lr = 0.5, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 3, wordNgrams = 2
0.68, 0.66:     lr = 0.5, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 3, wordNgrams = 3
0.67, 0.64:     lr = 0.5, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 4, wordNgrams = 1
0.65, 0.63:     lr = 0.5, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 4, wordNgrams = 2
0.64, 0.62:     lr = 0.5, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 4, wordNgrams = 3
0.65, 0.63:     lr = 0.5, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 5, wordNgrams = 1
0.64, 0.62:     lr = 0.5, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 5, wordNgrams = 2
0.62, 0.62:     lr = 0.5, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 5, wordNgrams = 3
0.63, 0.6:     lr = 0.5, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 6, wordNgrams = 1
0.62, 0.6:     lr = 0.5, dim = 5, ws = 5, epoch = 1, minn = 3, maxn = 6, wordNgrams = 2
0.61, 0.59:     lr = 0.

`0.83, 0.79: *** lr = 0.95, dim = 20, ws = 5, epoch = 1, minn = 3, maxn = 6, wordNgrams = 3`

In [397]:
all_data_path = os.path.join('./', 'sentiment.all')

sentiment_model_all = train_supervised(
    input=all_data_path,
    lr=0.95,
    dim=20,
    ws=5,
    epoch=1,
    minCount=1,
    minCountLabel=0,
    minn=3,
    maxn=6,
    neg=5,
    wordNgrams=3,
    loss='softmax',
    bucket=200000,
    lrUpdateRate=100,
    t=1e-4,
    label="__label__",
    verbose=2,
    pretrainedVectors=""
)
print_results(*sentiment_model_all.test(all_data_path))
sentiment_model_all.save_model("sentiment_model_all.bin")

N	2748
P@1	0.795
R@1	0.795


** word vectors **

In [320]:
sentiment_model_all.get_word_vector("cute")

array([-0.00696967, -0.00622005,  0.00869602, -0.00219836,  0.00484758,
        0.00945656,  0.0034168 ,  0.0121602 , -0.00564961, -0.00165913,
       -0.0096696 , -0.0029134 , -0.00033705, -0.00801563, -0.00356869,
        0.00047487, -0.0075628 , -0.00981004, -0.00897263,  0.01880032], dtype=float32)

In [337]:
sentiment_model_all.get_sentence_vector("nice item")

array([ 0.01921101, -0.00310891, -0.01493727, -0.03274819,  0.00308804,
       -0.00818889,  0.04900367, -0.04045218, -0.02591317, -0.00249004,
       -0.00122899,  0.03255024,  0.01380752,  0.02900477, -0.00446767,
        0.00078815,  0.00281659,  0.00531309,  0.00241404, -0.02496254], dtype=float32)

In [328]:
??sentiment_model_all.get_sentence_vector()

In [347]:
sentiment_model_all.get_subwords("high quality")

(['<hi',
  '<hig',
  '<high',
  '<high ',
  'hig',
  'high',
  'high ',
  'high q',
  'igh',
  'igh ',
  'igh q',
  'igh qu',
  'gh ',
  'gh q',
  'gh qu',
  'gh qua',
  'h q',
  'h qu',
  'h qua',
  'h qual',
  ' qu',
  ' qua',
  ' qual',
  ' quali',
  'qua',
  'qual',
  'quali',
  'qualit',
  'ual',
  'uali',
  'ualit',
  'uality',
  'ali',
  'alit',
  'ality',
  'ality>',
  'lit',
  'lity',
  'lity>',
  'ity',
  'ity>',
  'ty>'],
 array([ 64551, 179660, 144286,  42140,  35950, 148644,  85966, 155465,
        137226, 204512, 140607,  23998, 182437, 169092, 202335,  81922,
         84435, 117954,  65525,  61421,  98772, 190411, 104063, 114122,
         61317,  14493, 140276,  14498,  27332, 146947, 149663,  38522,
         85874, 203604, 105187, 124621, 184669,  76804,  33216,  36274,
        128470,  71191]))

In [332]:
??sentiment_model_all.get_subwords("cute")

In [339]:
words = sentiment_model_all.get_words()
for w in words[0:10]:
    print("{}  --> {}".format(w, sentiment_model_all.get_word_vector(w)))

</s>  --> [-0.04100632  0.04522638 -0.01677407  0.00292806  0.0229811   0.00338773
  0.00756884  0.02253154  0.06442698  0.04932172 -0.01690026  0.00282984
  0.04310306  0.02014819 -0.01655152 -0.01285936 -0.00584081  0.02349056
  0.00740045 -0.02155281]
the  --> [-0.0071901   0.01994328  0.02624204  0.03745943 -0.00059376 -0.00149082
 -0.06161203  0.05379729  0.0401204   0.01964574 -0.02374848 -0.01760563
 -0.01573259 -0.02479217 -0.03392079 -0.0255501   0.01105041 -0.00880719
  0.00618013  0.02632367]
and  --> [ 0.0916687  -0.05802166 -0.08009838 -0.14427777  0.01681603 -0.00543345
  0.22197804 -0.24706686 -0.15971978 -0.08586492  0.02742781  0.11533955
  0.06565166  0.11805582  0.01800646 -0.01872155 -0.03583391  0.01949752
 -0.00679308 -0.21350285]
a  --> [ 0.00700946  0.02059963  0.02920412  0.00793921 -0.00057545  0.01418647
 -0.02106544  0.03127332  0.05109321  0.01088802 -0.05195991 -0.00320869
  0.00074375 -0.01648562  0.00107517 -0.00479806 -0.00248752  0.00536313
 -0.0107729

### Check out the following for full set of functionalities you can get from fasttext

'https://fasttext.cc/docs/en/unsupervised-tutorial.html'

In case you need a smaller model to deploy to edge device:

In [315]:
sentiment_model_all.quantize(input=all_data_path, qnorm=True, retrain=True, cutoff=200000)
print_results(*sentiment_model_all.test(all_data_path))
sentiment_model_all.save_model("sentiment_model_all.ftz")

N	2748
P@1	0.900
R@1	0.900


In [348]:
from fastText import train_unsupervised

In [350]:
??train_unsupervised